In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

torch.cuda.empty_cache()

In [2]:
path='/kaggle/working/Cherry'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!mkdir /kaggle/working/Cherry

!cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/test/Cherry* /kaggle/working/Cherry
!cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/train/Cherry* /kaggle/working/Cherry

transfrom = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
    ])

dataset=ImageFolder(path,transform=transfrom)

train_dataset,test_dataset=train_test_split(dataset,test_size=0.2,shuffle=True,random_state=43)

batch_size = 32
n_iters = 10000
num_epochs = n_iters / (len(train_dataset) / batch_size)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
from torch.autograd import Variable
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.cnn = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(8*112*112, 2)
        
        
    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
model = NeuralNetwork()

model = model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(int(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.cuda())
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss 0.6641663908958435. Accuracy: 53.50819778442383
Iteration: 1000. Loss 0.5154607892036438. Accuracy: 76.13114929199219
Iteration: 1500. Loss 0.5964817404747009. Accuracy: 76.59016418457031
Iteration: 2000. Loss 0.5543346405029297. Accuracy: 75.08197021484375
Iteration: 2500. Loss 0.3476760685443878. Accuracy: 84.7868881225586
Iteration: 3000. Loss 0.24991948902606964. Accuracy: 87.40983581542969
Iteration: 3500. Loss 0.2410769760608673. Accuracy: 95.08197021484375
Iteration: 4000. Loss 0.5059701800346375. Accuracy: 80.7213134765625
Iteration: 4500. Loss 0.1018894612789154. Accuracy: 95.47541046142578
Iteration: 5000. Loss 0.17721909284591675. Accuracy: 85.57376861572266
Iteration: 5500. Loss 0.09634353965520859. Accuracy: 93.90164184570312
Iteration: 6000. Loss 0.0308147631585598. Accuracy: 95.60655975341797
Iteration: 6500. Loss 0.08792216330766678. Accuracy: 94.29508209228516
Iteration: 7000. Loss 0.05256728082895279. Accuracy: 96.26229858398438
Iteration: 7500. L